# Pytorch
## 加载数据
dataset -> dataloader

In [ ]:
from torch.utils.data import Dataset
from PIL import Image
import os

class my_data(Dataset):

    def __init__(self, root_dir, label_dir) -> None:
        super().__init__()
        self.root_dir = root_dir
        self.label_dir = label_dir
        self.path = os.path.join(self.root_dir, self.label_dir)
        self.img_path = os.listdir(self.path)
    
    def __getitem__(self, index):
        img_name = self.img_path[index]
        img_item_path = os.path.join(self.root_dir, self.label_dir, img_name)
        img = Image.open(img_item_path)
        label = self.label_dir
        return img, label
    
    def __len__(self):
        return len(self.img_path)

root_dir = '/Users/chen/Desktop/练手数据集/train'
ants_image_dir = 'ants_image'
ants_dataset = my_data(root_dir, ants_image_dir)

img, label = ants_dataset[0]
img

# len(ants_dataset)

## Tensorboard and Transform

### 使用tensorboard
### 使用transform转换成tensor数据格式

In [ ]:
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from PIL import Image


writer = SummaryWriter('/Users/chen/Desktop/练手数据集/train/logs')
image_path = ('/Users/chen/Desktop/练手数据集/train/ants_image/0013035.jpg')
image_PIL = Image.open(image_path)
img_array = np.array(image_PIL)

writer.add_image('test', img_array, 2,dataformats="HWC")
# for i in range(100):

#     writer.add_scalar('y=x',3*i,i)  #tag, scalar_value(y), global_step(x)


writer.close()

In [ ]:
# 使用tensorboard

from PIL import Image
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms

img_path = '/Users/chen/Desktop/练手数据集/train/ants_image/5650366_e22b7e1065.jpg'
img = Image.open(img_path)

writer = SummaryWriter('/Users/chen/Desktop/练手数据集/train/logs')

# toTensor
tensor_trans = transforms.ToTensor()
ten_img = tensor_trans(img)

writer.add_image('tor',ten_img)

writer.close()

In [ ]:
from torchvision import transforms
from PIL import Image

img = Image.open('/Users/chen/Desktop/练手数据集/train/ants_image/6240329_72c01e663e.jpg')

# normalize
print(ten_img[0][0][0])
tran = transforms.Normalize([0.5,1,0.1],[0.2,0.3,1])
img_nor = tran(ten_img)
print(img_nor[0][0][0])

writer.add_image('Nor',img_nor)

writer.close()



In [ ]:
# resize - PIL
print(img.size)
trans_size = transforms.Resize((512,512))
img_resized = trans_size(img)
print(img_resized.size)

# PIL - resize - toTensor
img_resized = trans_size(img)
print(img_resized.size)
type(img_resized)

img_resized = tensor_trans(img_resized)
type(img_resized)

# compose 连续调用方法集合
trans_resized2 = transforms.Resize(800)
trans_compose = transforms.Compose([trans_resized2,tensor_trans])
img_resized2 = trans_compose(img)

writer.add_image('resized2',img_resized2)

writer.close()


### 使用torchvision中的数据集

In [ ]:
import torchvision

train_set = torchvision.datasets.CIFAR10(root='./dataset',train=True,download=True)
test_set = torchvision.datasets.CIFAR10(root='./dataset',train=False,download=True)

train_set[0]


In [ ]:
from turtle import forward
from torch import nn
import torch

class Tudui(nn.Module):
    def __init__(self) -> None:
        super().__init__()

    def forward(self,input):
        output = input + 1
        return output

tudui = Tudui()
x = torch.tensor(1.0)
opt = tudui(x)
opt

## Convolutional layers

In [ ]:
import torch
import torch.nn.functional as F

input = torch.tensor(
    [
        [1,2,0,3,1],
        [0,1,2,3,1],
        [1,2,1,0,0],
        [5,2,3,1,1],
        [2,1,0,1,1]
    ]
)

kernel = torch.tensor(
    [
        [1,2,1],
        [0,1,0],
        [2,1,0]
    ]
)

input = torch.reshape(input,(1,1,5,5))
kernel = torch.reshape(kernel,(1,1,3,3))

print(input.shape)
print(kernel.shape)

output = F.conv2d(input,kernel,stride = 1)
print(output)

output2 = F.conv2d(input,kernel,stride = 2)
print(output2)

output3 = F.conv2d(input,kernel,stride = 1,padding=1)
print(output3)



In [ ]:
import torchvision
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.nn import Conv2d
from torch.utils.tensorboard import SummaryWriter


dataset = torchvision.datasets.CIFAR10('./dataset',train=False,transform=torchvision.transforms.ToTensor(),download=True)
dataloader = DataLoader(dataset,batch_size=64)

class Tudui(nn.Module):
    def __init__(self) -> None:
        super(Tudui,self).__init__()
        self.conv1 = Conv2d(in_channels=3,out_channels=6,kernel_size=3,stride=1,padding=0)
    
    def forward(self,x):
        x = self.conv1(x)
        return x

tudui = Tudui()
step = 0
writer = SummaryWriter('./logs')
for data in dataloader:
    imgs, targets = data
    output = tudui(imgs)
    writer.add_images('input',imgs,step)
    output = torch.reshape(output,(-1,3,30,30))
    writer.add_images('output',output,step)

    step += 1


## Pooling layer
maxpool

In [ ]:
import torch
import torchvision
from torch import nn
from torch.nn import MaxPool2d
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

dataset = torchvision.datasets.CIFAR10("./dataset", train=False, download=True,
                                       transform=torchvision.transforms.ToTensor())

dataloader = DataLoader(dataset, batch_size=64)

class Tudui(nn.Module):
    def __init__(self):
        super(Tudui, self).__init__()
        self.maxpool1 = MaxPool2d(kernel_size=3, ceil_mode=False)

    def forward(self, input):
        output = self.maxpool1(input)
        return output

tudui = Tudui()

writer = SummaryWriter("./logs_maxpool")
step = 0

for data in dataloader:
    imgs, targets = data
    writer.add_images("input", imgs, step)
    output = tudui(imgs)
    writer.add_images("output", output, step)
    step = step + 1

writer.close()



## No linear activation / Sigmoid

In [ ]:
import torch
from torch import nn
from torch.nn import ReLU
input = torch.tensor(
    [
        [1,-0.5],
        [-1,3]
    ]
)

output = torch.reshape(input,(-1,1,2,2))

output.shape

class Tudui(nn.Module):
    def __init__(self) -> None:
        super(Tudui,self).__init__()
        self.relu1 = ReLU()
    
    def forward(self,input):
        output = self.relu1(input)
        return output

tudui = Tudui()
output = tudui(input)
print(output)

In [ ]:
import torch
import torchvision
from torch import nn
from torch.nn import MaxPool2d
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.nn import Sigmoid

dataset = torchvision.datasets.CIFAR10("./dataset", train=False, download=True,transform=torchvision.transforms.ToTensor())
dataloader = DataLoader(dataset, batch_size=64)


class Tudui(nn.Module):
    def __init__(self) -> None:
        super(Tudui,self).__init__()
        self.sigmoid1 = Sigmoid()
    
    def forward(self,input):
        output = self.sigmoid1(input)
        return output

tudui = Tudui()

writer = SummaryWriter('./logs_sigmoid')
step = 0
for data in dataloader:
    imgs, targets = data
    writer.add_images('input',imgs,global_step=step)
    output=tudui(imgs)
    writer.add_images('output',output,step)
    step += 1

writer.close()


## Linear layer

In [ ]:
import torch
import torchvision
from torch import nn
from torch.nn import Linear
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.nn import Sigmoid

dataset = torchvision.datasets.CIFAR10("./dataset", train=False, download=True,transform=torchvision.transforms.ToTensor())
dataloader = DataLoader(dataset, batch_size=64)

class Tudui(nn.Module):
    def __init__(self) -> None:
        super(Tudui,self).__init__()
        self.linear1 = Linear(196688,10)
        
    def forward(self,input):
        output = self.linear1(input)
        return output

tudui = Tudui()

for data in dataloader:
    imgs, targets = data
    output = torch.reshape(imgs,(1,1,1,-1))
    output = tudui(output)
    print(output.shape)

## Sequential

In [ ]:
from torch import nn
from torch.nn import Conv2d,MaxPool2d,Flatten,Linear,Sequential
import torch
from torch.utils.tensorboard import SummaryWriter

class Tudui(nn.Module):
    def __init__(self) -> None:
        super(Tudui,self).__init__()
        self.model1 = Sequential(
            Conv2d(3,32,5,padding=2),
            MaxPool2d(2),
            Conv2d(32,32,5,padding=2),
            MaxPool2d(2),
            Conv2d(32,64,5,padding=2),
            MaxPool2d(2),
            Flatten(),
            Linear(1024,64),
            Linear(64,10)
        )

    def forward(self,x):
        x = self.model1(x)
        return x

writer = SummaryWriter('./logs_seq')
tudui = Tudui()

input = torch.ones((64,3,32,32))
output = tudui(input)
writer.add_graph(tudui,input)
writer.close()


## Loss function / Backwards

In [ ]:
from multiprocessing import reduction
import torch
from torch.nn import L1Loss,MSELoss
from torch import nn

input = torch.tensor([1,2,3],dtype=float)
target = torch.tensor([1,2,5],dtype=float)

# inputs = torch.reshape(input,(1,1,1,3))
# targets = torch.reshape(target,(1,1,1,3))

loss = L1Loss(reduction='sum')
result = loss(input,target)

print(result)

loss2 = MSELoss()
result2 = loss2(input,target)

print(result2)

x = torch.tensor([0.1,0.2,0.3])
y = torch.tensor([1])
x = torch.reshape(x,(1,3))
loss_cross = nn.CrossEntropyLoss()
result3 = loss_cross(x,y)
result3

In [ ]:
from torch import nn
from torch.nn import Conv2d,MaxPool2d,Flatten,Linear,Sequential
import torch
from torch.utils.tensorboard import SummaryWriter
import torchvision
from torch.utils.data import DataLoader


dataset = torchvision.datasets.CIFAR10("./dataset", train=False, download=True,transform=torchvision.transforms.ToTensor())
dataloader = DataLoader(dataset, batch_size=3)

class Tudui(nn.Module):
    def __init__(self) -> None:
        super(Tudui,self).__init__()
        self.model1 = Sequential(
            Conv2d(3,32,5,padding=2),
            MaxPool2d(2),
            Conv2d(32,32,5,padding=2),
            MaxPool2d(2),
            Conv2d(32,64,5,padding=2),
            MaxPool2d(2),
            Flatten(),
            Linear(1024,64),
            Linear(64,10)
        )

    def forward(self,x):
        x = self.model1(x)
        return x

tudui = Tudui()
loss = nn.CrossEntropyLoss()
for data in dataloader:
    imgs, targets = data
    output = tudui(imgs)
    result_loss = loss(output,targets)
    result_loss.backward()
    print(result_loss)

## Optimizer

In [ ]:
from torch import nn
from torch.nn import Conv2d,MaxPool2d,Flatten,Linear,Sequential
import torch
from torch.utils.tensorboard import SummaryWriter
import torchvision
from torch.utils.data import DataLoader


dataset = torchvision.datasets.CIFAR10("./dataset", train=False, download=True,transform=torchvision.transforms.ToTensor())
dataloader = DataLoader(dataset, batch_size=3)

class Tudui(nn.Module):
    def __init__(self) -> None:
        super(Tudui,self).__init__()
        self.model1 = Sequential(
            Conv2d(3,32,5,padding=2),
            MaxPool2d(2),
            Conv2d(32,32,5,padding=2),
            MaxPool2d(2),
            Conv2d(32,64,5,padding=2),
            MaxPool2d(2),
            Flatten(),
            Linear(1024,64),
            Linear(64,10)
        )

    def forward(self,x):
        x = self.model1(x)
        return x

tudui = Tudui()

loss = nn.CrossEntropyLoss()

optim = torch.optim.SGD(tudui.parameters(),lr=0.01)

for epoch in range(20):
    running_loss = 0.0
    for data in dataloader:
        imgs, targets = data
        output = tudui(imgs)
        result_loss = loss(output,targets)
        
        optim.zero_grad()
        result_loss.backward()
        optim.step()
        # print(result_loss)
        running_loss += result_loss
    print(running_loss)

## 完整的模型训练套路
- dataset
- dataloader
- models
- loss function
- optimizer
- variable
- start training


In [ ]:
import torchvision
from torch import nn
from torch.utils.tensorboard import SummaryWriter

# dataset
train_data = torchvision.datasets.CIFAR10("./dataset", train=True, download=True,transform=torchvision.transforms.ToTensor())
test_data = torchvision.datasets.CIFAR10("./dataset", train=False, download=True,transform=torchvision.transforms.ToTensor())

train_data_size = len(train_data)
test_data_size = len(test_data)

print(f'训练集长度{train_data_size}\n测试集长度{test_data_size}')

# dataloader
train_dataloader = DataLoader(train_data,batch_size=64)
test_dataloader = DataLoader(test_data,batch_size=64)

# create models
class Tudui(nn.Module):
    def __init__(self) -> None:
        super(Tudui,self).__init__()
        self.model = Sequential(
            Conv2d(3,32,5,padding=2),
            MaxPool2d(2),
            Conv2d(32,32,5,padding=2),
            MaxPool2d(2),
            Conv2d(32,64,5,padding=2),
            MaxPool2d(2),
            Flatten(),
            Linear(1024,64),
            Linear(64,10)
        )

    def forward(self,x):
        x = self.model(x)
        return x

tudui = Tudui()

# loss function
loss_fn = nn.CrossEntropyLoss()

# optimizer
learning_rate = 0.01
optimizer = torch.optim.SGD(tudui.parameters(),lr=learning_rate)

# variable
total_train_step = 0
total_test_step = 0
epoch = 10

# tensorboard
writer = SummaryWriter('./logs_final')


# training
for i in range(epoch):
    for data in train_dataloader:
        imgs, targets = data
        output = tudui(imgs)
        loss = loss_fn(output, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step += 1
        if total_train_step % 100 == 0:
            print(f'训练次数{total_train_step}, loss:{loss.item()}')
        writer.add_scalar('train_loss',loss.item(),total_train_step)

# testing
total_test_loss = 0
with torch.no_grad():
    for data in test_dataloader:
        imgs, targets = data
        output = tudui(imgs)
        loss = loss_fn(output, targets)
        total_test_loss += loss.item()
writer.add_scalar('train_loss',loss.item(),total_train_step)
# print(total_test_loss)

writer.close()
